<a href="https://colab.research.google.com/github/Vincenzo-Miracula/Disinformation-on-a-Network/blob/main/DisinformationOnNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import requests
from urllib.parse import urlparse
from tqdm import tqdm
import matplotlib.pyplot as plt
import networkx as nx
tqdm.pandas()

In [ ]:
df = pd.read_excel('C:/Users/vince/OneDrive/Desktop/Vincenzo/1.Dottorato/Tesi/Covid/Network-COVIDITA/Covid_tweet_SentimentEmotion_ita.xlsx')#, lines=True)
df.columns

Index(['Unnamed: 0', 'type', 'authorName', 'text', 'likeCount', 'createdAt',
       'inReplyToUsername', 'textClean', 'Sentiment', 'Emotion'],
      dtype='object')

In [ ]:
lista_dis = pd.read_excel('C:/Users/vince/OneDrive/Desktop/Vincenzo/2.py/Disinformation_on_Network/domain_list_clean.xlsx')
lista_disinformazione = lista_dis['url'].to_list()

# parte sui link

In [ ]:
# step1
def estrai_link(testo):
    match = re.search(r'(https?://[^/s]+)', testo)
    return match.group(0) if match else 'None'

df['link'] = df['text'].progress_apply(estrai_link)

  0%|          | 0/151127 [00:00<?, ?it/s]

100%|██████████| 151127/151127 [00:00<00:00, 835679.04it/s]


In [ ]:
filterDF = df.query("link != 'None'")

In [ ]:
len(df), len(filterDF)

In [ ]:
# step2
def url_completo(link):
  try:
      x = requests.get(link, allow_redirects=True, timeout=10)
      return x.url
  except requests.exceptions.RequestException:
    return "Dominio non attivo"

filterDF['url_completo'] = filterDF['link'].progress_apply(url_completo)

In [ ]:
# step3
def estrai_url_base(url):
    if url == "Dominio non attivo":
        return "Dominio non attivo"
    elif url.startswith("https://t.me/"):
        return url  #url forma originale di telegram
    elif "x.com/" in url:
        base_url = url.split('/')
        return f"https://x.com/{base_url[3]}/"
    elif url.startswith("https://www.facebook.com/"):
        # solo facebook.com/username
        parsed_url = urlparse(url)
        return f"{parsed_url.scheme}://{parsed_url.netloc}/{parsed_url.path.split('/')[1]}"
    else:
        return urlparse(url).hostname

filterDF['url_base'] = filterDF['url_completo'].progress_apply(estrai_url_base)

In [ ]:
# step4
def espandi_per_mention(df):
    righe_espanse = []

    # Barra di progresso per iterare sul DataFrame
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Espansione righe"):
        # Estrai solo menzioni valide (@ seguite da un nome utente valido, non URL come @https)
        menzioni = [m for m in re.findall(r'@(\w+)', row['text']) if not m.startswith('https')]

        # Crea una nuova riga per ogni menzione
        for menzione in menzioni:
            nuova_riga = row.copy()  # Copia la riga originale
            nuova_riga['rt'] = menzione  # Aggiungi la menzione nella colonna 'rt'
            righe_espanse.append(nuova_riga)

    # Restituisce un DataFrame espanso
    return pd.DataFrame(righe_espanse)

# Applica la funzione sul DataFrame originale
df_expanded = espandi_per_mention(filterDF)

# parte network

In [ ]:
# step5
def crea_grafo_relazioni(df, colonna_utenti, colonna_link_base, lista_disinformazione=[]):
    # Creiamo un grafo vuoto
    G = nx.Graph()

    # Mappa per ogni URL base e gli utenti che lo hanno condiviso
    link_base_to_users = {}

    # Iteriamo ogni riga per costruire la mappa degli URL e degli utenti
    for idx, row in df.iterrows():
        utente = row[colonna_utenti]
        link_base = row[colonna_link_base]

        # Verifica che l'URL non sia NaN
        if pd.notna(link_base):
            # Aggiungi l'utente alla lista di chi ha condiviso questo link
            if link_base not in link_base_to_users:
                link_base_to_users[link_base] = []
            link_base_to_users[link_base].append(utente)

    # Creiamo le relazioni tra gli utenti e assegniamo colori e status ai nodi
    for link_base, users in link_base_to_users.items():
        is_disi = link_base in lista_disinformazione  # Verifica se l'URL è sospetto
        node_color = 'red' if is_disi else 'blue'  # Colore del nodo
        node_status = 'sospetto' if is_disi else 'non sospetto'  # Stato del nodo

        # Aggiungiamo nodi per gli utenti con il colore e lo stato corretti
        for utente in users:
            if not G.has_node(utente):
                G.add_node(utente, color=node_color, status=node_status)

        # Creiamo gli archi tra gli utenti che hanno condiviso lo stesso link
        for i in range(len(users)):
            for j in range(i + 1, len(users)):
                u1, u2 = users[i], users[j]
                if not G.has_edge(u1, u2):  # Evita duplicati
                    G.add_edge(u1, u2)

    self_loops = list(nx.selfloop_edges(G))
    if self_loops:
        G.remove_edges_from(self_loops)
    # Visualizza il numero di nodi e archi nel grafo
    print(f'Numero di nodi: {G.number_of_nodes()}')
    print(f'Numero di archi: {G.number_of_edges()}')

    print("Network pronto. Ricordati di calcolare le misure ed esportarlo in Gephi")
    return G

In [ ]:
df_expanded = pd.read_excel('/content/dffitrato_CovidTweetIta.xlsx')
df_expanded.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'type', 'authorName', 'text', 'likeCount',
       'createdAt', 'inReplyToUsername', 'textClean', 'Sentiment', 'Emotion',
       'link', 'url_completo', 'url_base'],
      dtype='object')

In [ ]:
G = crea_grafo_relazioni(df_expanded, 'authorName', 'url_base', lista_disinformazione)

Numero di nodi: 373
Numero di archi: 565
Network pronto. Ricordati di calcolare le misure ed esportarlo in Gephi


In [ ]:
# step6
degree_centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, degree_centrality, 'degree_centrality')
# 2. Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(G)
nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')
# 3. Closeness Centrality
closeness_centrality = nx.closeness_centrality(G)
nx.set_node_attributes(G, closeness_centrality, 'closeness_centrality')
# 4. PageRank
pagerank = nx.pagerank(G)
nx.set_node_attributes(G, pagerank, 'pagerank')
# 5. Density network
density = nx.density(G)
print(f"Densità del grafo: {density}")

# Salviamo il grafo in formato GEXF con attributi
nx.write_gexf(G, 'test222.gexf')

Densità del grafo: 0.008143791980166624
